# Exploratory Data Analysis: Hands on Data
Today we will be demonstrating the following key exploratory data analysis techniques using an example dataset:

**Agenda:**
1. Importing libraries & packages
2. Importing tabular data to a DataFrame
3. Inspecting DataFrame structure
4. Concatenation
5. Renaming Columns
6. Exploring values
7. Exporting DataFrames
8. Merging
9. Plotting



## The Data
Our example dataset is daily summaries of air quality data from Providence, RI. It will give you some experience with working with temporal data.

The Rhode Island Department of Environmental Management (RIDEM) and Rhode Island Department of Health (RIDOH) collects air quality data at several sites across Rhode Island. We will be examining data from one site at the Community of Rhode Island (CCRI) Liston Campus. Here's some background:

* The CCRI site is part of the EPA's *State or Local Air Monitoring Stations* (SLAMS) and *National Air Toxics Trends Sites* (NATTS) networks.
* A variety of air pollutants (particulate matter (PM), volatile organic carbon (VOCs),  polycyclic aromatic hydrocarbons (PAHs), carbonyls, black carbon) have been monitored at this site since 2005.
* A reference for some of the dataset's [field descriptions](https://aqs.epa.gov/aqsweb/airdata/FileFormats.html#_daily_summary_files).
* The data was obtained from the Environmental Protection Agency (EPA) [Air Quality Data website](https://www.epa.gov/outdoor-air-quality-data).
        <div>
        <img src="./images/aq-site-info.png" width="400"/>
        </div>

We will use a subset of this data in the demonstrations below and give you a chance to work with a larger dataset during the hands-on lab.

*Links*
[EPA Air Quality Data Interactive Map](https://www.epa.gov/outdoor-air-quality-data/interactive-map-air-quality-monitors) - Data source
[RIDEM 2022 Annual Monitoring Report](https://dem.ri.gov/sites/g/files/xkgbur861/files/2023-01/airnet22.pdf) - More information about the site and other monitoring locations across the state.

---

## 1. Importing libraries & packages
Importing packages typically appears at the top of the file.
* `import <package_name>` is the most basic command
* The package can be imported with an alias to shorten verbosity. Common packages will often have a conventional alias.
<blockquote>

```python
import pandas
pandas.read_csv('path')

# VS as an alias

import pandas as pd
pd.read_csv('path')
```
</blockquote>


In [ ]:
import pandas as pd  # Import pandas library as an alias of 'pd'
# import matplotlib.pyplot as plt  # Import the sub-package pyplot from the matplotlib library as an alias of 'plt'
from pathlib import Path  # Import filesystem path package, for easier pathing to files and outputs

# Magic command for jupyter notebook to generate figures within the notebook
%matplotlib inline

## 2. Importing tabular data to a DataFrame
The pandas package reads tabular data into a data structure called a `DataFrame`. Some examples of read functions are below:
* [`pd.read_csv`](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) - Comma-delimited or other delimited files
* [`pd.read_fwf`](https://pandas.pydata.org/docs/reference/api/pandas.read_fwf.html#) - Fixed width files
* [`pd.read_excel`](https://pandas.pydata.org/docs/reference/api/pandas.read_excel.html) - Microsoft excel files
* [`pd.read_sql`](https://pandas.pydata.org/docs/reference/api/pandas.read_sql.html) - SQL query or database table
* See [pandas I/O documentation](https://pandas.pydata.org/docs/reference/io.html#input-output) for more examples

We will be working with the `pd.read_csv()` because our data is comma-delimited. This function defaults to read comma-delimited files, but can be used on any delimited text file when the seperator is specified.

A. To start we need specify the path to our data directory:
```
project
├── data
│   └── raw
│       └── monthly   <- Data is here
│
└── notebooks         <- Our working directory is here
```
We will be using package `os` and `Path` from `pathlib` to create out directory path because it standardizes pathing between operating systems. Path separators are different between Unix (Mac & Linux; using `/`) and Windows (using `\`) operating systems. Avoiding full string paths makes the code universal.

In [ ]:
# Create a path to the data directory. The `..` means go up one level from the current working directory
data_path = Path('..', 'data')

## We extend the path to the monthly data directory
path_to_monthly_data = data_path / 'raw' / 'monthly'
path_to_monthly_data2 = data_path.joinpath('raw',
                                           'monthly')  # Alternative syntax for extending path

print(f'This is the monthly data directory: {path_to_monthly_data}')

Using the path generated above, we will read the first month of data (January 2022).

In [ ]:
# Read and save the DataFrame object to a variable 'df_2022_01'
df_2022_01 = pd.read_csv(path_to_monthly_data / 'daily_44_007_0022_2022_01.csv')

## 3. Inspecting DataFrame Structure
Now that we have imported the data to a DataFrame. Some questions we are curious about:
1. Did it import correctly?
2. What does the table look like? Number of rows? Columns?
3. Do we need all the data we are importing?
4. Is the data in the correct format?

We can inspect the DataFrame object by looking at its **attributes** and using DataFrame **methods**.

Here are useful **attributes** of the dataframe
* `.shape`:  Table dimensions
* `.columns`:  Sequence of columns
* `.index`:  Sequence of row indexes/labels
* `.dtypes`: Data types by column

Here are a few useful **methods** to inspect a dataframe:
* `.head()`: Shows the first 5 rows--can change the number by supplying an integer.
* `.tail()`: Shows the last 5 rows--can change the number by supplying an integer.
* `.info()`: Combines several DataFrame attributes to one report.
* `.select_dtypes()`: Useful for viewing only columns of certain data types.


<div class="alert alert-block alert-info">
Python objects may have <b>attributes</b> and <b>methods</b>.

<b>attributes</b> - Are properties of the object type. Say that there is a `Person` object, the person's favorite food is one of their properties.
<b>methods</b> - Are functions bound to an object type. They often perform a process that uses the object's properties. A method of a `Person` object, could be report writing.

Attributes and methods are accessed by using dot (`.`) connectors to the object. The difference is that methods have `()` at the end so arguments can be passed.
>Example:
```
George.favorite_food  # Accessing an attribute
>>> 'Pho'
my_report = George.write_report(topic='favorite_food', pages=5)  # Calling a method
```
</div>

In [ ]:
df_2022_01.head()

In [ ]:
df_2022_01.tail()

In [ ]:
df_2022_01.shape

In [ ]:
df_2022_01.columns

In [ ]:
df_2022_01.info()

<div class="alert alert-block alert-info">
What is an "object" dtype?

<b>Short Answer:</b> It is a column of string or mixed data types (e.g. string, ints, floats, etc). Typically object dtype columns from an imported CSV will be a column of strings.

<b>Long Answer:</b>  Pandas was built upon the numpy package on its backend. Numpy can only store information in an array where each value is encoded in the same number of bytes. Because strings can be of variable length, they do not conform to the fixed byte requirement. Instead Pandas creates an object array with pointers to the strings and  the pointers are of equal byte size. This is similar for columns with mixtures of data types.
</div>

In [ ]:
# Inspect Numerical Fields
df_2022_01.select_dtypes(include=['int', 'float']).head(100)

In [ ]:
# Inspect Object fields
df_2022_01.select_dtypes(include='object').head(100)

**Back to our questions:**

1. Did it import correctly?
2. What does the table look like? Number of rows? Columns?
3. Do we need all the data we are importing?
4. Is the data in the correct format?

* There are many columns we could drop because they all have the same value such as: "Local Site Name" and "Address". We know we are only working with one site for this analysis so these columns don't provide much value. These columns are long string fields that take up more memory. Dropping them would improve performance if this dataset gets really large.
* The date would be more useful as a datetime data type rather than as string. This will allow for filtering by time and other useful datetime operations.

We can supply additional arguments to the `read_csv` function to handle these specifications.

In [ ]:
# Create a list of the columns we wish to keep
keep_cols = ['Parameter Code', 'POC', 'Parameter Name', 'Duration Description',
             'Pollutant Standard',
             'Date (Local)', 'Year', 'Day In Year (Local)', 'Units of Measure',
             'Exceptional Data Type',
             'Observation Count', 'Observation Percent', 'Arithmetic Mean', 'First Maximum Value',
             'First Maximum Hour', 'AQI', 'Daily Criteria Indicator', ]

# Read in the csv with additional arguments
df_2022_01_curated = pd.read_csv(path_to_monthly_data / 'daily_44_007_0022_2022_01.csv',
                                 usecols=keep_cols,  # Specify columns to keep
                                 parse_dates=['Date (Local)'],
                                 # Specify column to parse as a date instead of string
                                 date_format='%Y-%m-%d',  # Specify the format of date strings
                                 )
df_2022_01_curated.head()

In [ ]:
df_2022_01_curated.info()

Great! We've cut down the number of columns and converted the date field to a datetime format!
Next lets see how we can add more data from other files.

## 4. Concatenation
So far we've worked with one month's worth of data. Let's see how we can combine DataFrames together.

We will be using the [`pd.concat`](https://pandas.pydata.org/docs/reference/api/pandas.concat.html) function to combine two or more DataFrames.


In [ ]:
# Read in February data
df_2022_02_curated = pd.read_csv(path_to_monthly_data / 'daily_44_007_0022_2022_02.csv',
                                 usecols=keep_cols,  # Specify columns to keep
                                 parse_dates=['Date (Local)'],
                                 # Specify column to parse as a date instead of string
                                 date_format='%Y-%m-%d',  # Specify the format of date strings
                                 )
list_df_to_concat = [df_2022_01_curated, df_2022_02_curated]
df_combined = pd.concat(list_df_to_concat)
df_combined.head()

In [ ]:
df_combined.tail()

<div class="alert alert-block alert-warning">
After we concat we need to remember to reset the index because it does not do this automatically! We use the method `reset_index()` to do this.
</div>

In [ ]:
# Notice how the index is not the length of the dataframe?
df_combined.index

In [ ]:
# Resetting index
df_combined = df_combined.reset_index(drop=True)
df_combined.tail()

### Concat all the files!
Now that we've learned how to concatenate files. Let's combine all the monthly data.
Doing it manually for each file would be cumbersome. So lets use a function!

The function is not coded in this notebook but rather contained in the source code (src) directory. Why?

* This function seems like it could be useful in other analyses and scripts. Saving it in a common utility location gives us easy access in future analysis.


We've written this script already. It lives in the file at the directory path listed below.
```
project
├── src
│   └── data
│       └── utils.py   <- Function is saved here
```
We encourage you to take a look because it is a good example of writing a function covering with concepts of: 1) listing files in a directory, 2) for loops, and 3) if/else constructs combined with what we just learned about reading csv and concatenation.


<div class="alert alert-block alert-warning">
If we have not already, we need to install our src directory as a package to our environment to be able to access it.

`%pip install -e ..`

If we do it now we'll need to restart our kernel and run all cells to here before we continue.
</div>

In [ ]:
# Import the function from our source code directory
from src.data.utils import combine_csv_files

df_2022 = combine_csv_files(path_to_monthly_data,
                            prefix='daily_44_007_0022_2022_',
                            # Read in files only starting with this prefix
                            suffix='.csv',  # Read in files only ending with this suffix
                            usecols=keep_cols,  # Specify columns to keep
                            parse_dates=['Date (Local)'],
                            # Specify column to parse as a date instead of string
                            date_format='%Y-%m-%d', )  # Specify the format of date strings

In [ ]:
df_2022.shape

In [ ]:
df_2022.head()

In [ ]:
df_2022.tail()

## 5. Renaming Columns
Before we jump into more detailed EDA we'll want to update the DataFrame to make our lives a bit easier. We'll be typing column names often to query the data so lets start by renaming the columns a standard format.

1. lowercase
2. underscores instead of spaces
3. simplify complex names

Methods we'll use:
1. [`str.lower()`](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.lower.html): Lowercases the column names
2. [`str.replace()`](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.replace.html): Finds and replaces sub-strings
3. [`rename()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html): Renames columns or index labels



In [ ]:
# Remember we access columns with the columns attribute
df_2022.columns

In [ ]:
# We can cast a lowercase method across the columns with the `str.lower()` method
df_2022.columns.str.lower()

In [ ]:
# We cast a `str.replace()` method after the lowercase method. Though this starts to look hard to read.
df_2022.columns.str.lower().str.replace(' ', '_')

# A cleaner way is to wrap it in (), allowing us to separate the dot connectors to multiple lines
(df_2022.columns
 .str.lower()  # Lowercase the names
 .str.replace(' ', '_')  # Replace Spaces with underscores
 )

In [ ]:
# The above examples are output demonstrations. It doesn't actually change the dataframe's column until we assign the output.
print('---Before Assignment---')
print(df_2022.columns)

# Assigning reformatted column names to the DataFrame's columns attribute
df_2022.columns = (df_2022.columns
                   .str.lower()  # Lowercase the names
                   .str.replace(' ', '_')  # Replace Spaces with underscores
                   )
print('---After Assignment---')
print(df_2022.columns)

In [ ]:
# Use the rename method to rename specific columns
df_2022 = df_2022.rename(columns={'date_(local)': 'date',
                                  'day_in_year_(local)': 'day_in_year',
                                  }
                         )
df_2022.columns

Great! Now are column names are much more manageable.

## 6. Exploring Values
Let's start exploring the dataset.

Questions:
1. How complete is the data?
2. How many parameters are measured?
3. How often are the parameters measured?
4. What are the descriptive stats of the numeral data?

We will cover the following topics:
1. Checking for Nulls
2. Indexing
3. Counts and Uniques
4. Querying
5. Descriptive Stats and groupby

### 6.1 Checking for Nulls
Nulls or NAs are values that represent no data or missing data.
* They are often represented as **NaN** for "not a number"
* Pandas will typically fill NaNs for blank values upon import.
* Be aware that scientific datasets often use large negative numbers outside of the normal range (like -999) to represent null data.

We can check for the number of null values quickly using the method `isna()` and summing the results
* `isna()` will create a boolean matrix and `sum()` will sum by columns.


In [ ]:
df_2022.isna().sum()

Luckily this dataset does not have missing data in important fields. The fields with null values make sense.
* Not all chemicals measured have a regulatory standard that would be in the *pollutant_standard* field, only the most toxic.
* *exceptional_data_type* is a flag field for anomalous conditions and events.

Pandas has great documentation for working with and filling in missing data that we recommend reviewing. [Link](https://pandas.pydata.org/docs/user_guide/missing_data.html)

### 6.2 Indexing
We index a dataframe using brackets.

```
# For a single column
df_2022['parameter_name']

# For multiple columns
df_2022[['parameter_name', 'parameter_code']]

# For a slice of rows (same as list indexing)
df_2022[5:10]

# For a slice of rows and columns use the .loc method
df_2022.loc[5:10, ['parameter_name', 'parameter_code']]
```


In [ ]:
df_2022.loc[5:10, ['parameter_name', 'parameter_code']]

### 6.3 Counts and Uniques
Counts and unique values are great for looking at categorical data.

In [ ]:
# Get the count of unique values in each column
df_2022.nunique()

In [ ]:
# Get a list of unique values in a column
df_2022['parameter_name'].unique()

In [ ]:
# Get the counts of each parameter
counts = df_2022['parameter_name'].value_counts()
counts

The output is abbreviated. To see specific analytes you need index with a list.

In [ ]:
counts[['PM2.5 - Local Conditions', 'Lead PM10 STP', 'Acetaldehyde', 'Cyclohexane']]

The number of Particulate Matter <2.5 microns (PM2.5) measurements seems high. Let's take a closer look at this parameter.

### 6.4 Querying
There are two main ways to query a DataFrame:
1. Masking and Indexing
2. [`query()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.query.html) method

The choice depends on preference and context.
* Masking and Indxing is clearer for complex queries with multiple conditions.
* The `query()` method good for short and quick queries.

You can try either method below by comment/uncommenting lines 3 and 6.

In [ ]:
# Masking and Indexing
mask_param = df_2022['parameter_name'] == 'PM2.5 - Local Conditions'
df_2022[mask_param].head(12)

# Query method
# df_2022.query('parameter_name == "PM2.5 - Local Conditions"').head(12)

In [ ]:
df_2022.loc[mask_param, ['parameter_name', 'duration_description', 'pollutant_standard', 'arithmetic_mean', 'first_maximum_value', 'first_maximum_hour','aqi']].head(12)

*What do we notice about the PM2.5 measurement?*
* There are 3 records per day. They differ in duration_description and pollutant_standard.
* The "1 HOUR" duration seems to have the most accurate information for our purposes because has more "arithmetic_mean" precision, and better information about the first maximum value and hour over the day.
* The "1 Hour" duration records does not have Air Quality Index "aqi" information

**Conclusion**
* We should keep only the "1 HOUR" duration and remove the others.
* We'll save the other durations in a separate DataFrame in the case we want the daily AQI information.

We can do this with querying!

In [ ]:
# Create a mask for the values we want to remove
mask_remove = (df_2022['parameter_name'] == 'PM2.5 - Local Conditions') & (df_2022['duration_description'] != '1 HOUR')

# Create dataframe of PM2.5 records we removed
df_removed_records = df_2022[mask_remove].reset_index(drop=True)

# Create dataframe of everything we want to keep. The NOT (~) operator reverses the mask.
df_2022_cleaned_v1 = df_2022[~mask_remove].reset_index(drop=True)

In [ ]:
counts2 = df_2022_cleaned_v1['parameter_name'].value_counts()
counts2

In [ ]:
counts2['PM2.5 - Local Conditions']

#### 6.5 Descriptive Stats and groupby
The method `.describe()` is a great way to get descriptive statistics on numerical columns. When called on the dataframe, it will automatically provide statistics on each numerical column.

In [ ]:
df_2022_cleaned_v1.describe()

Though this isn't very useful for our dataset at the moment because it's creating statistics across 90+ parameters. It would be better to chunk the DataFrame by parameter and run describe. This is where a powerful method called `groupby` is useful.

In [ ]:
# Create a Groupby object with parameter_name
gb_param = df_2022_cleaned_v1.groupby(by='parameter_name')
gb_param['arithmetic_mean'].describe()

In [ ]:
# To look at the PM2.5 stats
df_gb_stats = gb_param['arithmetic_mean'].describe()
df_gb_stats.loc['PM2.5 - Local Conditions']

You can groupby multiple categories. Here's an example of sample counts by parameter and month.

In [ ]:
gb_param_month = df_2022_cleaned_v1.groupby(by=['parameter_name', df_2022_cleaned_v1['date'].dt.month])
df_param_monthly_counts = gb_param_month['arithmetic_mean'].count()
df_param_monthly_counts.loc[['PM2.5 - Local Conditions', '1,1,1,2-Tetrachloroethane']]

## 7. Exporting DataFrames
We made some interesting groupby tables with summary stats in the last section. Let's try exporting them to view in another application.

We use the `to_csv()` method to export to csv.
See the [Pandas documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_csv.html) to see all the other formats you can export to.

In [ ]:
# Create a Path object to the reports folder
outputs_path = Path('..', 'reports')

# Save the groupby parameters and descriptive stats of the arithmetic mean to a DataFrame
df_gb_param_mean_describe = gb_param['arithmetic_mean'].describe()

# Call the to_csv method on the dataframe specifying path and filename
df_gb_param_mean_describe.to_csv(outputs_path / 'stats-arithmetic_mean-by-param.csv')

## 8. Merging
If we wanted to join two tables based on common key we would use the [`merge()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html) method.

There is a file with parameter classes that categorizes the parameters. This might help us make sense of which parameters are related to each other. Lets join it to our dataframe!

In [ ]:
df_param_classes = pd.read_csv(data_path / 'raw' / 'params_class.csv')
df_param_classes.head()

In [ ]:
# Count of the number of parameters in each class
df_param_classes['class'].value_counts()

In [ ]:
# Merge the dataframes using parameter_code as the key with a left-join
df_2022_cleaned_v2 = df_2022_cleaned_v1.merge(df_param_classes, on='parameter_code', how='left')
df_2022_cleaned_v2.head()

We used a left-join here. This means that only keys from the left table matter in the join. Keys that are in the right table but not the left will not be joined.

Notice that parameter_name is duplicated with an appended "_x" and "_y" because the column existed in both tables. You can prevent this by indexing only the columns you want to join. For now we are just going to drop and rename the columns for an example of the `drop()` method!

In [ ]:
df_2022_cleaned_v3 = (df_2022_cleaned_v2
                      .drop(columns='parameter_name_y')  # Remove the column
                      .rename(columns={'parameter_name_x': 'parameter_name'})  # We've seen this method previously!
                      )
df_2022_cleaned_v3.columns

Let's take a look at the online documentation for this function. [`pd.read_csv`](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html)

 At the top is the function call signature:
>pandas.read_csv(filepath_or_buffer, *, sep=_NoDefault.no_default, delimiter=None, header='infer', ...)
* This demonstrates how to use the function in code with all the available arguments.
* There are two types of arguments: *Positional* and *Keyword*
    1. **Positional arguments** are listed first. They are required and need to be specified in order. In this example there is only one, `filepath_or_buffer`.
    2. **Keyword arguments** are listed after positional arguments and are optional. They have an `=` after the name to denote default values.

    Positional arguments do not need to be specified by name while keyword arguments must be specified by name.
    ```python
    # Both of these calls are acceptable
    pd.read_csv('data/raw/datafile.csv', sep=',')
    pd.read_csv(filepath_or_buffer='data/raw/datafile.csv', sep=',')
